In [1]:
# getting started
import pandas as pd
import geopandas as gpd
import folium

from shapely import geometry

In [2]:
# создаем сетку

data = gpd.read_file('OKN_pts.geojson').to_crs("EPSG:32637")

total_bounds = data.total_bounds
minX, minY, maxX, maxY = total_bounds

square_size = 500

In [3]:
grid_cells = []
x, y = (minX, minY)
geom_array = []

while y <= maxY:
        while x <= maxX:
            geom = geometry.Polygon([(x,y), (x, y+square_size), (x+square_size, y+square_size), (x+square_size, y), (x, y)])
            geom_array.append(geom)
            x += square_size
        x = minX
        y += square_size


fishnet = gpd.GeoDataFrame(geom_array, columns=['geometry']).set_crs('EPSG:32637')
fishnet['id'] = fishnet.index

fishnet.to_file('grid.gpkg', driver="GPKG")

In [4]:
merged = gpd.sjoin(data, fishnet, how='left', predicate='within')
merged['n'] = 1
dissolve = merged.dissolve(by="index_right", aggfunc="count")
fishnet.loc[dissolve.index, 'n'] = dissolve.n.values

In [5]:
data = data.to_crs('EPSG:4326')

In [6]:
m = folium.Map(location=[data.centroid.y.mean(), data.centroid.x.mean()], zoom_start=13,  tiles="cartodb positron", control_scale=True)

/var/folders/mc/_dkt15px7sb86y74_d18r5kc0000gn/T/ipykernel_16273/2256734257.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[data.centroid.y.mean(), data.centroid.x.mean()], zoom_start=13,  tiles="cartodb positron", control_scale=True)
/var/folders/mc/_dkt15px7sb86y74_d18r5kc0000gn/T/ipykernel_16273/2256734257.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[data.centroid.y.mean(), data.centroid.x.mean()], zoom_start=13,  tiles="cartodb positron", control_scale=True)


In [7]:
# кластеры точек

from folium.plugins import MarkerCluster

In [8]:
marker_cluster = MarkerCluster(name='Heritage Sites')
mc1= folium.plugins.FeatureGroupSubGroup(marker_cluster, 'Heritage Sites')
m.add_child(marker_cluster)
m.add_child(mc1)
mc1.add_child(folium.GeoJson(data.to_json(), embed=False, show=False))

In [9]:
# добавляем полигоны зданий ОКН

okn_buildings = open('OKN_poly.geojson').read()

folium.GeoJson(
    okn_buildings,
    name='ОКН здания',
    tooltip=folium.GeoJsonTooltip(fields=["name_2"]),
    popup=folium.GeoJsonPopup(fields=['name_2']),
    style_function=lambda x: {
        "fillColor": 'yellow'
    },
    highlight_function=lambda x: {"fillOpacity": 0.8},
    zoom_on_click=True,
    show=False,
).add_to(m)

In [10]:
# граница города

adm_border = open('Rostov_bound.geojson').read()

folium.GeoJson(
    adm_border,
    name='Граница города'
).add_to(m)

In [11]:
# монастыри

monasteries = open('monasteries.geojson').read()

folium.GeoJson(
    monasteries,
    name='Территория монастырей'
).add_to(m)

In [12]:
# добавляем сетку плотности на карту

folium.Choropleth(
    geo_data=fishnet,
    data=fishnet,
    columns=['id', 'n'],
    fill_color='YlGnBu',
    fill_opacity = 0.,
    key_on='id',
    nan_fill_opacity=0,
   line_color = "#0000",
   legend_name="Количество ОКН на 500м2",
   name='Концентрация ОКН'
).add_to(m)

In [13]:
# добавляем мишуру

In [14]:
folium.LayerControl().add_to(m)

In [15]:
from folium.plugins import MousePosition
from folium.plugins import Fullscreen
# from folium.plugins import FloatImage

In [16]:
MousePosition().add_to(m)
Fullscreen(
    position="bottomright",
    title="Полный экран",
    title_cancel="Выйти",
    force_separate_button=True,
).add_to(m)

In [17]:
#image = 
#FloatImage(image, bottom=40, left=65).add_to(m)

In [18]:
m.save('index.html')